## Import Libraries

Importing python libraries to be used in this data analysis:

* **sqlite3** for interacting with local relational database.
* **numpy** and **pandas** for data ingestion and manipulation.
* **matplotlib** for data visualization.
* **sklearn** for Machine Learning.
* **customplot** which contains custom functions written for this notebook.

In [19]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
# from customplot import *

<h1 style="font-size:2em;color:#2467C0">Data Acquisition</h1>

Command below read the dataset
Make sure the run the import cell above(shift + enter) before running ingest code below.

**df** is a variable pointing to a pandas data frame.

Remember to have the database and the jupyter notebook in the same folder

In [20]:
# Create your connection
con = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", con)

<h1 style="font-size:2em;color:#2467C0">Exploring Data</h1

Data exploration by generating simple statistics of the data

Checking data columns using a pandas attribute called "columns"

In [21]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

Displaying simple statistics of the dataset. Run each cell to make sure you see the outputs.

In [22]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,183978.0,91989.500000,53110.018250,1.0,45995.25,91989.5,137983.75,183978.0
player_fifa_api_id,183978.0,165671.524291,53851.094769,2.0,155798.00,183488.0,199848.00,234141.0
player_api_id,183978.0,135900.617324,136927.840510,2625.0,34763.00,77741.0,191080.00,750584.0
overall_rating,183142.0,68.600015,7.041139,33.0,64.00,69.0,73.00,94.0
potential,183142.0,73.460353,6.592271,39.0,69.00,74.0,78.00,97.0
crossing,183142.0,55.086883,17.242135,1.0,45.00,59.0,68.00,95.0
finishing,183142.0,49.921078,19.038705,1.0,34.00,53.0,65.00,97.0
heading_accuracy,183142.0,57.266023,16.488905,1.0,49.00,60.0,68.00,98.0
short_passing,183142.0,62.429672,14.194068,3.0,57.00,65.0,72.00,97.0
volleys,181265.0,49.468436,18.256618,1.0,35.00,52.0,64.00,93.0


<h1 style="font-size:2em;color:#2467C0">Data Cleaning: Handling Missing Data</h1>

Clean data by converting or getting rid of null or missing values. The next code cell shows if any of the 183978 rows have null value in one of the 42 columns

In [23]:
# Is any row NULL?
df.isnull().any().any(), df.shape

(True, (183978, 42))

Next code cell tries to find out how many data points in each column are null.

In [24]:
df.isnull().sum(axis=0)

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

# Fixing Null Values By Deleting Them

The next two lines drops the null values by going through each row.

In [25]:
# Take initial number of rows
rows = df.shape[0]

# Drop the NULL values
df = df.dropna()

In [26]:
# Check if all NULLL values are gone?
print(rows)
df.isnull().any().any(), df.shape

183978


(False, (180354, 42))

In [27]:
# How many rows with NULL values?
rows - df.shape[0]

3624

The rows can be randomly shuffled so as not to have the same 

In [28]:
#Shuffle the rows of df so we get a distributed sample when we display top few rows

df = df.reindex(np.random.permutation(df.index))

<h1 style="font-size:2em;color:#2467C0">Predicting: 'Overall_Rating' of a player</h1>

The data is now clean giving room to better understanding and diving further into the dataset.

To take a look at the first few rows of the dataset; use the head function for data frames for this task.

This gives every column in every row.

In [29]:
df.head(5)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
181956,181957,185181,192064,2010-02-22 00:00:00,61.0,74.0,right,medium,medium,21.0,...,54.0,51.0,21.0,21.0,11.0,60.0,61.0,60.0,57.0,65.0
98558,98559,201332,188503,2015-04-24 00:00:00,57.0,62.0,right,medium,medium,52.0,...,61.0,57.0,50.0,54.0,47.0,8.0,6.0,7.0,7.0,9.0
165055,165056,187043,70078,2013-08-23 00:00:00,73.0,80.0,left,medium,medium,58.0,...,73.0,42.0,58.0,54.0,42.0,6.0,15.0,6.0,8.0,13.0
100583,100584,205976,358127,2014-07-18 00:00:00,75.0,80.0,right,high,high,60.0,...,65.0,64.0,25.0,23.0,25.0,7.0,13.0,8.0,5.0,8.0
76051,76052,186130,163200,2016-04-14 00:00:00,72.0,73.0,right,medium,medium,54.0,...,49.0,52.0,73.0,76.0,76.0,11.0,12.0,12.0,14.0,13.0


In the case you are interested in plotting some columns, use pandas column selection option as follows. Ignore the first column in the output of the one line code below. It is the unique identifier that acts as an index for the data.

In [30]:
df[:10][['penalties', 'overall_rating']]

,penalties,overall_rating
181956,51.0,61.0
98558,57.0,57.0
165055,42.0,73.0
100583,64.0,75.0
76051,52.0,72.0
147054,11.0,69.0
101969,53.0,54.0
84062,49.0,75.0
78441,62.0,65.0
79695,42.0,56.0


### Feature Correlation Analysis

Check if "penalties" is correlated to "overall_rating". To do this, use a similar selection operation, but for all rows and within the correlation function.

# Pearson's Correlation Coefficient


In [31]:
# Are these correlated ?
df['overall_rating'].corr(df['penalties'])

0.3927151079111884

Pearson's Correlation Coefficient for the two columns is 0.39

Pearson goes from -1 to +1. A value of 0 shows no correlation. A value greater than 0 shows a positive correlation; as the value of one variable increases, so does the other variable. A value less than 0 indicates a negative correlation; as the value of one variable increases, the value of the other variable decreases.

A value of 0.39 shows some correlation but it could be stronger.

Now that the attributes are slightly correlated, it gives hope that a meaningful predictor can be built using this 'weakly' correlated features.

Now, create a list of features that will iterate the same operation on.

## Create a list of potential Features that you want to measure correlation with

In [32]:
potentialFeatures = ['acceleration', 'curve', 'free_kick_accuracy', 'ball_control', 'shot_power', 'stamina']

The for loop below prints out the correlation coeeficient of "overall_rating" of a player with each feature added to the list as potential

In [33]:
for f in potentialFeatures:
    related = df['overall_rating'].corr(df[f])
    print("%s: %f" %(f, related))

acceleration: 0.243998
curve: 0.357566
free_kick_accuracy: 0.349800
ball_control: 0.443991
shot_power: 0.428053
stamina: 0.325606


**"ball_control"(0.44)** and **"shot_power"(0.43)** are features that have the highest correlation coefficient with "overall_rating"